In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

22/07/23 23:59:54 WARN Utils: Your hostname, kuchoco-GP62-6QE resolves to a loopback address: 127.0.1.1; using 192.168.219.113 instead (on interface wlp2s0)
22/07/23 23:59:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/23 23:59:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_green = spark.read.parquet('data/origin_green/')

In [3]:
df_green.rdd.take(2) # RDD

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2019, 12, 19, 0, 52, 30), lpep_dropoff_datetime=datetime.datetime(2019, 12, 19, 0, 54, 39), store_and_fwd_flag='N', RatecodeID=1.0, PULocationID=264, DOLocationID=264, passenger_count=5.0, trip_distance=0.0, fare_amount=3.5, extra=0.5, mta_tax=0.5, tip_amount=0.01, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=4.81, payment_type=1.0, trip_type=1.0, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 9, 45, 58), lpep_dropoff_datetime=datetime.datetime(2020, 1, 1, 9, 56, 39), store_and_fwd_flag='N', RatecodeID=5.0, PULocationID=66, DOLocationID=65, passenger_count=2.0, trip_distance=1.28, fare_amount=20.0, extra=0.0, mta_tax=0.0, tip_amount=4.06, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=24.36, payment_type=1.0, trip_type=2.0, congestion_surcharge=0.0)]

In [5]:
df_green.take(2)  

list

```sql
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour,
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    hour, zone 
ORDER BY
    hour, zone
```

In [14]:
RDD = df_green\
    .select('lpep_pickup_datetime', 'PULocationID' ,'total_amount')\
    .rdd

In [19]:
from datetime import datetime
start = datetime(2020,1,1)

print( RDD.filter(lambda row: row.lpep_pickup_datetime >= start).take(1) )

####### Using Function ########

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

print( RDD.filter(filter_outliers).take(1) )

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 9, 45, 58), PULocationID=66, total_amount=24.36)]
[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 1, 9, 45, 58), PULocationID=66, total_amount=24.36)]


In [39]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)

    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)


In [37]:

RDD \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)


## DateTime, LocationID, sum(amount), count 완성

[((datetime.datetime(2020, 1, 1, 9, 0), 42), (799.7600000000002, 52)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 255), (666.34, 28)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 59), (50.900000000000006, 3)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 36), (295.34000000000003, 11)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 37), (175.67, 6)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 146), (99.37, 6)),
 ((datetime.datetime(2020, 1, 1, 10, 0), 43), (21.76, 2)),
 ((datetime.datetime(2020, 1, 1, 10, 0), 82), (498.41000000000025, 39)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 197), (72.01, 3)),
 ((datetime.datetime(2020, 1, 1, 9, 0), 177), (274.95, 10))]

## To DataFrame

In [47]:
from collections import namedtuple

RevenuewRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])


def unwrap(row):
    return RevenuewRow(
        hour = row[0][0],
        zone = row[0][1],
        revenue = row[1][0],
        count = row[1][1])

In [49]:
df_result = RDD \
        .filter(filter_outliers) \
        .map(prepare_for_grouping) \
        .reduceByKey(calculate_revenue) \
        .map(unwrap) \
        .toDF()

In [50]:
df_result.schema

StructType(List(StructField(hour,TimestampType,true),StructField(zone,LongType,true),StructField(revenue,DoubleType,true),StructField(count,LongType,true)))

In [52]:
from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour',types.TimestampType(), True),
    types.StructField('zone',types.IntegerType(), True),
    types.StructField('revenue',types.DoubleType(), True),
    types.StructField('count',types.IntegerType(), True)
    ])


In [53]:
df_result = RDD \
        .filter(filter_outliers) \
        .map(prepare_for_grouping) \
        .reduceByKey(calculate_revenue) \
        .map(unwrap) \
        .toDF(result_schema)

In [54]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-01 09:00:00|  42| 799.7600000000002|   52|
|2020-01-01 09:00:00| 255|            666.34|   28|
|2020-01-01 09:00:00|  59|50.900000000000006|    3|
|2020-01-01 09:00:00|  36|295.34000000000003|   11|
|2020-01-01 09:00:00|  37|            175.67|    6|
|2020-01-01 09:00:00| 146|             99.37|    6|
|2020-01-01 10:00:00|  43|             21.76|    2|
|2020-01-01 10:00:00|  82|498.41000000000025|   39|
|2020-01-01 09:00:00| 197|             72.01|    3|
|2020-01-01 09:00:00| 177|            274.95|   10|
|2020-01-01 09:00:00| 196|             37.91|    2|
|2020-01-01 09:00:00| 258|              45.3|    1|
|2020-01-01 09:00:00|  29|              61.3|    1|
|2020-01-01 08:00:00| 116|              6.81|    1|
|2020-01-01 09:00:00|  85|             64.76|    4|
|2020-01-01 10:00:00| 226|228.91000000000005|   14|
|2020-01-01 

In [55]:
df_result.schema

StructType(List(StructField(hour,TimestampType,true),StructField(zone,IntegerType,true),StructField(revenue,DoubleType,true),StructField(count,IntegerType,true)))

In [56]:
df_result.write.parquet('tmp/green-revenue') 